In [26]:
import joblib
import optuna
import os,sys,inspect
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mne.datasets.sleep_physionet.age import fetch_data
import mne
import pandas as pd
from scipy import signal
from scipy.fftpack import fft

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

from src.features.build_features import break_2_bands, channels_fft

In [5]:
files = fetch_data(subjects=[0], recording=[1])

Using default location ~/mne_data for PHYSIONET_SLEEP...


In [6]:
psg = files[0][0]
anno = files[0][1]

In [27]:
raw = mne.io.read_raw_edf(psg, preload=True, stim_channel=None)

sampling_rate = raw.info['sfreq']

df = raw.to_data_frame(scalings=100.0)[['EEG Fpz-Cz', 'EEG Pz-Oz']]
# Preprocessing

df1 = break_2_bands(df)

df2 = channels_fft(df)

df1.set_index(np.arange(len(df1)))
df2.set_index(np.arange(len(df2)))

Extracting EDF parameters from /home/orel.ts@staff.technion.ac.il/mne_data/physionet-sleep-data/SC4001E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7949999  =      0.000 ... 79499.990 secs...


,EEG Fpz-Cz_fft,EEG Pz-Oz_fft
0,131.576720,278.928752
1,1.139181,1.322771
2,0.938774,0.834076
3,0.596252,0.426689
4,0.163376,2.080166
...,...,...
7949995,1.078992,1.178490
7949996,0.163376,2.080166
7949997,0.596252,0.426689
7949998,0.938774,0.834076


In [28]:
#########################
#   PLOT FFT of 5 bands 
#########################
Fs = 100
T = 1/Fs
N = len(df1)
xf = np.linspace(0.0, 1.0/(2.0*T), N//2)

bands_str = ["delta", "theta", "alpha", "sigma", "beta"]
fig = plt.figure()
for band in range(len(df1.columns)):
    ax = fig.add_subplot(2, 5, band+1)
    ffted = abs(fft(df1.iloc[:, band].values))
    ax.set_title(df1.columns[band].split("_")[0] + "-" + bands_str[band % 5])
    ax.plot(xf, ffted[:N//2], 'C'+ str(band % 5))
    
plt.show()

In [32]:
#########################################
#   PLOT sleeping events of MNE data 
#########################################
annot_train = mne.read_annotations(anno)

annotation_desc_2_event_id = {'Sleep stage W': 1,
                              'Sleep stage 1': 2,
                              'Sleep stage 2': 3,
                              'Sleep stage 3': 4,
                              'Sleep stage 4': 4,
                              'Sleep stage R': 5}

# keep last 30-min wake events before sleep and first 30-min wake events after
# sleep and redefine annotations on raw data
annot_train.crop(annot_train[1]['onset'] - 30 * 60,
                 annot_train[-2]['onset'] + 30 * 60)
raw.set_annotations(annot_train, emit_warning=False)

events_train, _ = mne.events_from_annotations(
    raw, event_id = annotation_desc_2_event_id, chunk_duration=30.)

# create a new event_id that unifies stages 3 and 4
event_id = {'Sleep stage W': 1,
            'Sleep stage 1': 2,
            'Sleep stage 2': 3,
            'Sleep stage 3/4': 4,
            'Sleep stage R': 5}

# plot events
fig = mne.viz.plot_events(events_train, event_id=event_id,
                          sfreq=raw.info['sfreq'],
                          first_samp=events_train[0, 0])

# keep the color-code for further plotting
stage_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 4', 'Sleep stage R', 'Sleep stage W']


In [ ]:
# data = np.abs(data["x"])
# coms = []
# for yf in data:
#     Fs = 100
#     T = 1/Fs
#     N = yf.shape[0]
#     xf = np.linspace(0.0, 1.0/(2.0*T), N//2)
#     y = yf[:, 0]

#     com = 0
#     for index, _ in enumerate(xf):
#         com += xf[index] * y[index]

#     com /= sum(y)
#     coms.append(com)
    
# fig, ax = plt.subplots()
# ax.plot(coms)
# plt.show()

In [ ]:
# fig, ax = plt.subplots()
# ax.plot(xf, np.abs(y[:N//2]))
# plt.show()

In [ ]:
# study = joblib.load('SC4001E0.pkl')
# df = study.trials_dataframe().drop(['state', 'datetime_start', 'datetime_complete'], axis=1)
# print(optuna.visualization.is_available())

In [ ]:
# fig = optuna.visualization.plot_optimization_history(study)
# fig.show()